# Application of Deep Knockoffs for Functional Magnetic Resonance Imaging to Generate Surrogate Data

This notebook describes the pipeline to follow to build Knockoffs from fMRI images. These knockoffs generate surrogate data that is used in non-parametric tests to obtain a Statistical Parametric Map (SPM) of the brain.

First of all, some things to consider:
* fMRI images data set we use in this work was made available by the Medical Image Processing Laboratory (MIPLAB) 
* The Knockoff library can be found at https://github.com/msesia/deepknockoffs.git


## Structure
The process is divided into 4 main parts, which consist of:
#### 1. Performing the **General Linear Model (GLM)** on the data
This is the classical method to obtain the SPM, which returns the fitted beta values for $$ y = X\beta$$ where y is the fMRI timecourse and X is the design matrix of the experiment.


#### 2. Generating Knockoffs
Given the data, the algorithm will build a machine to generate surrogate timecourses. 
There are three methods:
* Gaussian Knockoffs
* Low Rank Knockoffs
* Deep Knockoffs
    
    
#### 3. Performing **Non-Parametric Tests**
The GLM is applied to the generated surrogate data to get the beta values and these are used to threshold the true betas using Non-Parametric Tests, which can be:
* Uncorrected Non-Parametric Test
* Corrected Non-Parametric Test
    
#### 4. Visualizing the results
* Brain plot (Matlab)
* Confusion matrix




In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import seaborn as sns
matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)

from implementation import glm, knockoff_class, params
from implementation.load import load_pickle, load_fmri
from implementation.utils import KNOCK_DIR, compare_diagnostics

## 1. GLM

In [ ]:
# running the GLM on all tasks
glm.run()

 MOTOR 
Loading data for task MOTOR...
Loaded Data - Shape: (100, 379, 284)
Loaded Task Paradigms - Shape: (100, 284)
Computing GLM for task MOTOR...
Separating conditions...
Done!
Convolving...
Done!
Fitting GLM for 100 subjects and 379 regions...
Done!
Saving activations and beta values for task MOTOR...
 GAMBLING 
Loading data for task GAMBLING...
Loaded Data - Shape: (100, 379, 253)
Loaded Task Paradigms - Shape: (100, 253)
Computing GLM for task GAMBLING...
Separating conditions...
Done!
Convolving...
Done!
Fitting GLM for 100 subjects and 379 regions...
Done!
Saving activations and beta values for task GAMBLING...
 RELATIONAL 
Loading data for task RELATIONAL...
Loaded Data - Shape: (100, 379, 232)
Loaded Task Paradigms - Shape: (100, 232)
Computing GLM for task RELATIONAL...
Separating conditions...
Done!
Convolving...
Done!
Fitting GLM for 100 subjects and 379 regions...
Done!
Saving activations and beta values for task RELATIONAL...
 SOCIAL 
Loading data for task SOCIAL...
Loa

## 2. Building Knockoffs

In [ ]:
# selecting the same task and subject as in the report
task = 'MOTOR'
subject = 1

# loading all the data
fmri_data = load_fmri(task=task)

### a) Gaussian Knockoffs

In [ ]:
gaussian = knockoff_class.GaussianKnockOff(task, subject)  

gaussian.load_fmri()

# Pre-processing the data: clustering to decrease correlations
gaussian.pre_process(max_corr=.3)

# Training the machine to build second-order knockoffs
_ = gaussian.fit() 

# Plotting diagnostics to show the performance of the knockoffs
res_gaus = gaussian.diagnostics()

In [ ]:
# Building the knockoffs: the resulting data matrix will be of shape (real+knockoff, timepoints, regions),
# with (0,:,:) containing the real beta value and (1:, :, :) containing the knockoffs
data_gaussian = gaussian.transform()
print(data_gaussian.shape)

### b) Low Rank Knockoffs

In [ ]:
lowrank = knockoff_class.LowRankKnockOff(task, subject)  

lowrank.load_fmri()

# Training the machine to build low-ranked knockoffs
_ = lowrank.fit(rank=120) 

# Plotting diagnostics to show the performance of the knockoffs
res_lowrank = lowrank.diagnostics()

In [ ]:
# Building the knockoffs: the resulting data matrix will be of shape (real+knockoff, timepoints, regions),
# with (0,:,:) containing the real beta value and (1:, :, :) containing the knockoffs
data_lowrank = lowrank.transform()
print(data_lowrank.shape)

### c) Deep Knockoffs

We can load a previously trained machine or train a new one.

* **Loading a machine**

In [ ]:
# We can load a previously trained machine
deepko = knockoff_class.DeepKnockOff(task, subject)
deepko.pre_process(max_corr=.3, save=True)

# Loading previously trained machine
_, x_train = load_pickle(KNOCK_DIR, f'GaussianKO_tfMRI_t{task}_s{subject}_c0.3.pickle')
groups, _ = load_pickle(KNOCK_DIR, f'GaussianKO_mapping_t{task}_s{subject}_c0.3.pickle')
params = load_pickle(KNOCK_DIR, f'DeepKO_tMOTOR_s{subject}_params')

deepko.load_x(x_train)
deepko.load_params(params)
deepko.load_machine()

res_deepko = deepko.diagnostics()

In [ ]:
data_deepko = deepko.transform(groups=groups)

deepko_betas = deepko.statistic(data_deepko, save=True)
uncorrected_betas_deepko, corrected_betas_deepko = deepko.threshold(deepko_betas, save=True)

* **Training a new machine (Takes several hours 🥵🥵🥵)**

In [ ]:
# Or train a new knockoff machine
deepko = knockoff_class.DeepKnockOff(task, subject)  

# Pre-processing the data: clustering to avoid correlations
deepko.pre_process(max_corr=.3, save=True)

# Training the machine to build second-order knockoffs. The parameters can be changed at params.py
_ = deepko.fit() 

# Plotting diagnostics to show the performance of the knockoffs
res_deepko = deepko.diagnostics()

In [ ]:
# generating deep knockoffs
data_deepko = deepko.transform(groups=groups)
# calculating the GLM betas for the knockoffs
deepko_betas = deepko.statistic(data_deepko, save=True)
# executing the non-parametric test
uncorrected_betas_deepko, corrected_betas_deepko = deepko.threshold(deepko_betas, save=True)

In [ ]:
# Building the knockoffs: the resulting data matrix will be of shape (real+knockoff, timepoints, regions),
# with (0,:,:) containing the real beta value and (1:, :, :) containing the knockoffs
deepko.transform()
print(data_deepko.shape)

## Comparing the knockoff diagnostics

In [ ]:
# comparing the diagnostics for the knockoffs
res_total = pd.concat([res_gaus, res_lowrank, res_deepko], ignore_index=True, sort=False)
compare_diagnostics(res_total)

## 3. Non-Parametric tests

In [ ]:
# Computing the beta values of the knockoffs
gaussian_betas = gaussian.statistic(data_gaussian, save=True)
lowrank_betas = lowrank.statistic(data_lowrank, save=True)
deepko_betas = deepko.statistic(data_deepko, save=True)

In [ ]:
# Performing corrected and uncorrected Non-Parametric tests to threshold
uncorrected_betas_gaussian, corrected_betas_gaussian = gaussian.threshold(gaussian_betas, save=True)
uncorrected_betas_lowrank, corrected_betas_lowrank = lowrank.threshold(lowrank_betas, save=True)
uncorrected_betas_deepko, corrected_betas_deepko = deepko.threshold(deepko_betas, save=True)

## 4. Visualizing our results
The results can either be visualized in Matlab using the script `PlotGraph/our_plots.m`, which will show a brain plot with the activation regions; or by a confusion matrix, which gives a comparison on the active regions of the brain for each method compared to the GLM.

In [ ]:
# "true" betas computed with GLM
TRUE_BETAS = f'./data/output/beta/GLM_betas_{task}.mat'
# "true" controlled betas computed with GLM
TRUE_BETAS_CONTROLLED = f'./data/output/beta/GLM_controlled_betas_{task}.mat'
# "true" uncontrolled betas computed with GLM
TRUE_BETAS_UNCONTROLLED = f'./data/output/beta/GLM_uncontrolled_betas_{task}.mat'

true_betas = scipy.io.loadmat(TRUE_BETAS)['beta'][subject, :, :]  
true_betas_controlled = scipy.io.loadmat(TRUE_BETAS_CONTROLLED)['beta'][subject, :, :]  
true_betas_uncontrolled = scipy.io.loadmat(TRUE_BETAS_UNCONTROLLED)['beta'][subject, :, :] 

In [ ]:
def plot_confusion_matrix(true, methods, true_label, methods_labels):
    """
    Plots the confusion matrices given in the report.
    true: "true" GLM labels
    methods: betas to compare to the "true" GLM labels
    true_labels: label for the "true" beta values
    methods_labels: labels for the compared beta values    
    """
    true = [0 if x == 0 else 1 for x in true.ravel()]

    fig, axes = plt.subplots(1, len(methods), figsize=(15,3))
    fig.suptitle('Comparison')
    
    for idx, method in enumerate(methods):
        method = [0 if x == 0 else 1 for x in method.ravel()]
        data = {true_label:    true,
        methods_labels[idx]: method
        }

        df = pd.DataFrame(data, columns=[true_label, methods_labels[idx]])

        confusion_matrix = pd.crosstab(df[true_label], df[methods_labels[idx]], rownames=[true_label], 
                                       colnames=[methods_labels[idx]], margins=False)
    
        sns.heatmap(confusion_matrix, ax=axes[idx], annot=True, cmap = 'BuPu',fmt='g')
    
    plt.show()

true = true_betas_controlled
methods = [corrected_betas_gaussian, corrected_betas_lowrank, corrected_betas_deepko]
methods_labels = ['Gaussian', 'Low Rank', 'Deep Knockoffs']
true_label = 'GLM'
plot_confusion_matrix(true, methods, true_label, methods_labels)    

A sample brain plot:
<img src="data/output/img/DeepKO_corrected_betas_tMOTOR_s1_cond2.png">